# Case Study: Concrete Compressive Strength Prediction

In this case study, you'll solve a machine learning problem from start to finish.

The previous notebook, "analyze toydata", deals with a similar problem and can serve as a guideline for this exercise. You may also want to have a look at the [cheat sheet](https://franziskahorn.de/mlws_resources/cheatsheet.pdf) for more ideas and a concise overview of the relevant steps when developing a machine learning solution in any data science project. 

Feel free to get creative! 

### The Task

You are a data scientist at a startup that wants to build an app to help construction workers mix concrete of a certain target compressive strength (= the main quality parameter of concrete). The construction worker can enter the amounts of the different ingredients:
```
Cement:            _____ kg/m^3
Slag:              _____ kg/m^3
Fly Ash:           _____ kg/m^3
Fine Aggregate:    _____ kg/m^3
Coarse Aggregate:  _____ kg/m^3
Plasticizer:       _____ kg/m^3
Water:             _____ kg/m^3
```
and the app then tells them what the estimated compressive strength of this mixture will be:
```
Predicted Compressive Strength after 28 days: XX.X MPa
```
**Your job is it to build an ML model that predicts this compressive strength.**

Your colleagues have already prepared the backend for the app (see notebook `b`), which will load your model to make the predictions. Additionally, the backend also contains code to optimize the water content in the concrete recipe so that the mixture will get closer to the specified target strength:
```
Target strength:   _____ MPa

Recommended water amount: XX.X kg/m^3
New predicted strength with optimized water amount: XX.X MPa
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import joblib
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# these "magic commands" are helpful if you plan to import functions from another script
# where you keep changing things, i.e., if you change a function in the script
# it will automagically be reloaded in the notebook so you work with the latest version
%load_ext autoreload
%autoreload 2

## Step 0: Load the data

The original data can be obtained from the [UCI ML data repository](https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength). (If you're having trouble loading the xls file, you can also open it in Excel and then save it as a CSV file and use `pd.read_csv` instead.)

One data point in the dataset corresponds to one concrete mixture with the following variables:
- Mixture components (unit: kg in a m^3 mixture):
    - Cement
    - Blast Furnace Slag
    - Fly Ash
    - Water
    - Superplasticizer
    - Coarse Aggregate
    - Fine Aggregate
- Age (number of days the concrete mixture hardened before the compressive strength was measured)
- Concrete compressive strength (unit: MPa; the main quality indicator for concrete (i.e., how strong it is))

Our goal is to **predict the compressive strength** of a concrete mixture based on the amounts of the different components. We will later filter the data for only the 28-day measurements, since this is the most important measurement to determine whether the concrete is within the norm constraints and okay to be used for construction.

In [ ]:
# load the file into a dataframe with pandas
df = pd.read_excel("../data/Concrete_Data.xls")
# look at the raw data (first 5 rows)
df.head()

In [ ]:
# the column names are not super convenient for an analysis, so let's rename them
RENAME_DICT = {
    'Cement (component 1)(kg in a m^3 mixture)': "cement",
    'Blast Furnace Slag (component 2)(kg in a m^3 mixture)': "slag",
    'Fly Ash (component 3)(kg in a m^3 mixture)': "fly_ash",
    'Water  (component 4)(kg in a m^3 mixture)': "water",
    'Superplasticizer (component 5)(kg in a m^3 mixture)': "plasticizer",
    'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': "coarse_aggregate",
    'Fine Aggregate (component 7)(kg in a m^3 mixture)': "fine_aggregate", 
    'Age (day)': "age",
    'Concrete compressive strength(MPa, megapascals) ': "strength",
}
df = df.rename(columns=RENAME_DICT)
df.head()

## Step 1: Exploratory Analysis

Get a better understanding of the data and the problem:
- How are the individual variables distributed?
- Are any variables correlated? 
- Do you observe any patterns between the input and target variables? Do these make sense or is anything surprising?
- Anything else you should take into account when preprocessing the data later for the supervised learning task?

In [ ]:
# TODO: create some plots to better understand the data


## Step 2: Predict the 28-day Compressive Strength

Now that you've become more familiar with the dataset, it's time to tackle the real task, i.e., predict the 28-day compressive strength of a concrete mixture.

An evaluation pipeline is already set up below using a "stupid baseline" (= predicting the mean). Your task is to improve upon the performance by trying, for exampe... 
- different [models](https://scikit-learn.org/stable/supervised_learning.html)
- different [preprocessing steps](https://scikit-learn.org/stable/modules/preprocessing.html) (e.g., transformations or feature engineering)
- [hyperparameter tuning](https://scikit-learn.org/stable/modules/grid_search.html)
- [ensemble models](https://scikit-learn.org/stable/modules/ensemble.html) (i.e., combining different models)

Get creative :-)

**Tip:** To use your model within the app's backend later (notebook `b`), it's important that your final model incl. all necessary preprocessing steps are combined in a single estimator object. This can be accomplished with sklearn's [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) function. If necessary, you could even write a [custom transformer](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156) to perform more fancy feature engineering steps than what is provided by sklearn.


In [ ]:
# the most important measurement to check whether the concrete adheres to
# the norm is after 28 days (-> almost half of our samples)
df["age"].value_counts()

In [ ]:
# for the prediction task we use only the 28-day samples 
df = df.loc[df["age"] == 28].reset_index(drop=True)
df

In [ ]:
# define features (all input variables except age) and target
# CAUTION: do not change this - the backend assumes these will be the inputs for your model
features = ["cement", "slag", "fly_ash", "water", "plasticizer", "coarse_aggregate", "fine_aggregate"]
target = "strength"

In [ ]:
# you might have noticed that there are a few sampes that have the exact 
# same feature values, but different strengths...
# we just assume that these were repeat measurements and take the average
df = df.groupby(features).mean().reset_index()
df

In [ ]:
# extract training and validation/test sets from this dataframe
# (normally, we would also set aside a final test set,
# but we only have very few samples here)
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [ ]:
def eval_model(model, X_train, y_train, X_test, y_test):
    """
    Function to evaluate a trained regressor: prints R^2 and mean absolute error.
    
    Inputs:
        - model: the trained model
        - X_train, y_train: the training data
        - X_test, y_test: the test data
    """
    print(f"R^2 on training data: {model.score(X_train, y_train):.3f}")
    print(f"R^2 on test data: {model.score(X_test, y_test):.3f}")
    print(f"MAE on training data: {mean_absolute_error(y_train, model.predict(X_train)):.3f}")
    print(f"MAE on test data: {mean_absolute_error(y_test, model.predict(X_test)):.3f}")

In [ ]:
# train a dummy model
model = DummyRegressor()
model = model.fit(X_train, y_train)
# evaluate the model
eval_model(model, X_train, y_train, X_test, y_test)

In [ ]:
# TODO: now it's up to you: try an actual model and get better predictions!


In [ ]:
# save your final model so it can be used by the backend
# CAUTION: your model must be a single estimator object that also includes all necessary preprocessing steps 
# (e.g., by using the `make_pipeline` function mentioned above) which works on the originally defined features
joblib.dump(model, "strength_model.pkl")

## Step 3: Interpret the model

After you've found a model that achieves a decent performance, try to understand what it's doing.
- Calculate the permutation feature importance to see which features are most influential overall
- For the most important features, look at the partial dependence plot to see _how_ these features influence the outcome

Do these results make sense in terms of the actual physical process?

In [ ]:
# TODO: permutation feature importance


In [ ]:
# TODO: partial dependence plots


## Step 4: Use the backend

Head over to notebook `b` and run it from top to bottom and leave it open. It runs the server with our backend using your trained and saved model to make predictions.

In the code below we query the backend with a single data point to see what it returns.

In [ ]:
# url where the backend is running
port = 8000
backend_url = f"http://localhost:{port}/predict"

In [ ]:
# get one row from our feature matrix and convert it to a dictionary
x = X_test.iloc[0].to_dict()
# add a target strength (optional)
x["target_strength"] = 42.5
# send this as a json to the backend via a POST request
response = requests.post(backend_url, json=x)
# the result will also be dictionary with
# - the original water content
# - the predicted strength with the original water content
# - the optimized water content
# - the new predicted strength when using the optimized water content
result = response.json()
result

## Step 5: Optimization & What-If Analysis

When mixing concrete, water can usually be dosed rather flexibly.

Let's say our goal is to achieve a compressive strength after 28 days of 42.5 MPa.

Your prediction model is used in the backend to check whether the proposed concrete mixture would get too strong or too weak and then the water amount is adapted accordingly to get closer to the target.

Does your model help to get the production more on target?

In [ ]:
def plot_what_if(X, y, target_strength=42.5):
    """
    Compute and plot the optimized the water content for all data points.
    This code creates two plots:
        1. What-if results after optimization: by changing the water content, the strength predictions
           should be closer to the target strength, even after correcting for prediction errors.
           The legend includes the MATD = mean absolute target deviation, i.e., how far away the
           respective points are from the target strength on average.
        2. Original and optimized water content and resulting strength increase/decrease.
    
    Inputs:
        - X: pandas dataframe with original input features for all data points
        - y: pandas dataframe with true compressive strength values for all data points
        - target_strength: what we would like the output to be (default: 42.5)
    """
    # run the optimization for all data points
    water_org_s, water_new_s, pred_org_s, pred_new_s = [], [], [], []
    for i in range(len(X)):
        if not i % 10:
            print(f"backend queried for {i:2} / {len(X)} data points", end="\r")
        x = X.iloc[i].to_dict()
        x["target_strength"] = target_strength
        result = requests.post(backend_url, json=x).json()
        water_org_s.append(result["water_org"])
        water_new_s.append(result["water_new"])
        pred_org_s.append(result["pred_org"])
        pred_new_s.append(result["pred_new"])
    print(f"backend queried for {len(X)} / {len(X)} data points")
    # convert lists to numpy arrays for easier plotting
    water_org_s, water_new_s = np.array(water_org_s), np.array(water_new_s)
    pred_org_s, pred_new_s = np.array(pred_org_s), np.array(pred_new_s)
    # convert y to a numpy array to make sure the indices match up with the other arrays
    target_org_s = y.to_numpy()
    
    # plot the optimization results
    plt.figure(figsize=(10, 5))
    # dashed line to indicated the target value we wanted
    plt.hlines(target_strength, 0, len(water_org_s), "k", "dashed", linewidth=1)
    # original target values as light blue dots
    plt.plot(target_org_s, "o", c="#53DDFE", alpha=0.8, 
             label=f"original y (MATD: {np.abs(target_org_s - target_strength).mean():.1f})")
    # predicted target values with original water content as blue x
    plt.plot(pred_org_s, "x", c="#007693", 
             label=f"predicted y (MATD: {np.abs(pred_org_s - target_strength).mean():.1f})")
    # predicted target values with optimized water content as orange x
    plt.plot(pred_new_s, "x", c="#A84801", 
             label=f"optimized predicted y (MATD: {np.abs(pred_new_s - target_strength).mean():.1f})")
    # since our original predictions are not perfect, we shift our optimized predictions
    # by the error we made on the original predictions -> plot as orange dots
    pred_new_corrected_s = pred_new_s + (target_org_s - pred_org_s)
    plt.plot(pred_new_corrected_s, "o", alpha=0.8, c="#FE9C53", 
             label=f"realistic optimized y (MATD: {np.abs(pred_new_corrected_s - target_strength).mean():.1f})")
    plt.xticks([], [])
    plt.xlabel("samples")
    plt.ylabel("compressive strength [MPa]")
    plt.title("What-If Analysis")
    plt.legend(loc=2, bbox_to_anchor=(1.02, 1), numpoints=1)
    
    # plot original and optimized water content
    plt.figure()
    # diagonal line -> original and optimized water content are the same
    plt.plot([water_new_s.min(), water_new_s.max()], [water_new_s.min(), water_new_s.max()], "k", alpha=0.5)
    # points above the line: more water than before
    # points below the line: less water than before
    # color of dot shows whether the optimization resulted in a reduction or increase in predicted strength
    plt.scatter(water_org_s, water_new_s, c=pred_new_s-pred_org_s)
    plt.colorbar()
    plt.xlabel("original water content")
    plt.ylabel("optimized water content")
    plt.title("changes in water & strength");
    

In [ ]:
# run the optimization on the test set and plot the results
plot_what_if(X_test, y_test)

## Step 6: Presentation of results
Clean up your code & think about which results you want to present and the story they tell:
- What have you learned about concrete production and how is this reflected in the data?
- What is the best model that you found & its performance?
- Which preprocessing steps had the most impact on the performance (for different models)?
- Which features were the most influential and how did they impact the model's prediction?
- What have you learned in this case study? Did any of the results surprise you?